In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [12]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [13]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.12925447523593903
epoch:  1, loss: 0.08665452897548676
epoch:  2, loss: 0.06314680725336075
epoch:  3, loss: 0.05012567341327667
epoch:  4, loss: 0.04297208786010742
epoch:  5, loss: 0.039122775197029114
epoch:  6, loss: 0.03706199675798416
epoch:  7, loss: 0.03596305474638939
epoch:  8, loss: 0.03537771478295326
epoch:  9, loss: 0.03506619483232498
epoch:  10, loss: 0.0349002443253994
epoch:  11, loss: 0.03481151908636093
epoch:  12, loss: 0.034763749688863754
epoch:  13, loss: 0.034737709909677505
epoch:  14, loss: 0.0347231887280941
epoch:  15, loss: 0.03471479192376137
epoch:  16, loss: 0.03470965474843979
epoch:  17, loss: 0.0347028449177742
epoch:  18, loss: 0.03469572961330414
epoch:  19, loss: 0.034695591777563095
epoch:  20, loss: 0.03468509763479233
epoch:  21, loss: 0.034678954631090164
epoch:  22, loss: 0.03467718884348869
epoch:  23, loss: 0.03466811403632164
epoch:  24, loss: 0.03466273099184036
epoch:  25, loss: 0.034659452736377716
epoch:  26, loss: 0

In [14]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.8295341648094239
Test metrics:  R2 = 0.8281815633024978


In [15]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.1158432587981224
epoch:  1, loss: 0.07378571480512619
epoch:  2, loss: 0.05309215933084488
epoch:  3, loss: 0.043004605919122696
epoch:  4, loss: 0.03814656287431717
epoch:  5, loss: 0.03583122789859772
epoch:  6, loss: 0.03473467007279396
epoch:  7, loss: 0.03421512246131897
epoch:  8, loss: 0.0339660719037056
epoch:  9, loss: 0.03384292125701904
epoch:  10, loss: 0.03377803787589073
epoch:  11, loss: 0.03373997285962105
epoch:  12, loss: 0.03366062417626381
epoch:  13, loss: 0.03358980268239975
epoch:  14, loss: 0.03358781710267067
epoch:  15, loss: 0.03344167768955231
epoch:  16, loss: 0.033360887318849564
epoch:  17, loss: 0.033277180045843124
epoch:  18, loss: 0.03311292082071304
epoch:  19, loss: 0.03302168473601341
epoch:  20, loss: 0.03295879065990448
epoch:  21, loss: 0.032783523201942444
epoch:  22, loss: 0.03269325941801071
epoch:  23, loss: 0.03264087811112404
epoch:  24, loss: 0.03253110870718956
epoch:  25, loss: 0.032434042543172836
epoch:  26, loss: 0

In [16]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7428101024457374
Test metrics:  R2 = 0.7435365749712606
